<a href="https://colab.research.google.com/github/myazdani/hacking-minGPT/blob/main/play_seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/myazdani/hacking-minGPT.git

import sys
  
# adding Folder_2 to the system path
sys.path.insert(0, './hacking-minGPT/')

Cloning into 'hacking-minGPT'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 98 (delta 61), reused 62 (delta 29), pack-reused 0
Unpacking objects: 100% (98/98), done.


In [2]:
## only needed if running on google colab
!wget https://raw.githubusercontent.com/myazdani/attention-layer-demo/master/utils.py

--2022-07-10 06:29:58--  https://raw.githubusercontent.com/myazdani/attention-layer-demo/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3583 (3.5K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   3.50K  --.-KB/s    in 0s      

2022-07-10 06:29:59 (40.4 MB/s) - ‘utils.py’ saved [3583/3583]



In [3]:
import utils

data_gen = utils.DataUtil(seq_height_min = 1.0, 
                          seq_height_max = 25.0,
                          seq_width_min = 5.0, 
                          seq_width_max = 11.0,
                          seq_length = 100,
                          group_by_locations = False
                         )

In [4]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [5]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [6]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [7]:
train_input, train_targets, train_tr, train_bx = data_gen.generate_sequences(25000)
test_input, test_targets, test_tr, test_bx = data_gen.generate_sequences(1000)

In [8]:
train_input[0]

tensor([[ 1.3881e-01,  6.3907e-04,  3.5653e-02, -1.0254e-01, -1.4928e-02,
         -9.7536e-02, -8.0082e-02,  1.4810e-01, -4.3043e-02, -1.1965e-01,
         -6.2651e-03,  8.9815e-02, -6.1008e-03, -1.2954e-01, -1.2650e-01,
          5.6362e-02,  1.8373e+01,  1.8453e+01,  1.8570e+01,  1.8533e+01,
          1.8387e+01,  1.8613e+01,  6.8205e-02, -1.0463e-01,  2.1756e-01,
          3.4020e+00,  6.7923e+00,  9.9844e+00,  1.3520e+01,  1.6784e+01,
          1.6550e+01,  1.3446e+01,  1.0037e+01,  6.8722e+00,  3.4742e+00,
          1.3080e-01,  7.2165e-02, -3.5473e-02,  9.8057e-02,  1.3551e-01,
          1.4653e-01,  1.1693e+00,  2.7657e+00,  4.2827e+00,  5.9463e+00,
          6.4351e+00,  5.1312e+00,  3.8046e+00,  2.0117e+00,  5.1997e-01,
         -8.3284e-02, -2.7345e-02,  1.3278e-01, -3.3797e-02,  5.6181e-03,
          8.7424e-03, -2.9240e-02,  8.0360e-02, -6.2150e-02,  3.8748e-02,
          1.1138e-01,  1.4885e-01, -8.2257e-02, -5.1634e-02, -1.2590e-01,
          1.4295e-01, -7.3140e-02, -3.

In [9]:
train_input[0].size()

torch.Size([1, 100])

In [17]:
# get random 5 vals per seq and stack them all up 
pluck_vals = lambda x: x[0,torch.randperm(100)[:5]]
random_vals = torch.cat([pluck_vals(x) for x in train_input]).numpy()
print(random_vals.shape)

(125000,)


In [18]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=100, random_state=0).fit(random_vals[:,np.newaxis])
kmeans.labels_


kmeans.cluster_centers_.shape

(100, 1)

In [25]:
tarin_numpy = train_input[:5,:].view(-1,1).numpy()

In [29]:
torch.from_numpy(kmeans.predict(tarin_numpy)).view(5,-1).float()

tensor([[63.,  0., 76., 24.,  0., 24., 24., 63., 72., 93.,  0., 29.,  0., 93.,
         93., 76., 89.,  6.,  6.,  6., 89.,  6., 29., 24., 63.,  4., 92., 26.,
         49., 59., 98., 49., 26., 92.,  4., 63., 29., 72., 29., 63., 63., 37.,
         50., 39., 46., 23., 44., 86., 60., 82., 24., 72., 63., 72.,  0.,  0.,
         72., 29., 72., 76., 63., 63., 24., 72., 93., 63., 24., 72., 35., 35.,
         35., 35., 35., 35., 22., 24., 29., 93., 76., 93., 72., 63., 29., 29.,
          0., 76., 72., 72., 72., 76., 93., 63., 76., 29.,  0., 76.,  0., 76.,
         63., 24.],
        [76.,  0., 76., 63., 76., 93.,  0., 72., 63., 93.,  0., 63., 29., 29.,
         76., 93., 93., 93., 24.,  0., 29., 63., 89., 38., 38., 38., 89., 89.,
         89., 24., 29., 76., 72.,  0., 63., 43., 18., 81., 39.,  4., 34., 37.,
         24., 63., 29., 63., 24., 76., 76., 29.,  0., 24., 76., 76., 24.,  0.,
         24., 29., 76., 18., 88., 12., 77., 56., 88., 60., 93., 76., 24., 24.,
         54., 54.,  1., 54., 54.

In [51]:
from torch.utils.data import Dataset

class SeqDataset(Dataset):
    """
    wrap up seq dataset into our own, which will convert floating points into sequences of integers
    """
    
    def __init__(self, inputs, targets, kmeans_clusters):
        self.inputs = inputs
        self.targets = targets
        self.kmeans_clusters = kmeans_clusters
        
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x, y = self.inputs[idx], self.targets[idx]
        batch_size = x.shape[0]
        x_np = x.view(-1,1).numpy()
        y_np = y.view(-1,1).numpy()
        x_clusters = torch.from_numpy(self.kmeans_clusters.predict(x_np)).view(batch_size,-1).long()
        y_clusters = torch.from_numpy(self.kmeans_clusters.predict(y_np)).view(batch_size,-1).long()

        return x_clusters.squeeze(), y_clusters.squeeze()

In [52]:
train_dataset = SeqDataset(train_input, train_targets, kmeans)
test_dataset = SeqDataset(test_input, test_targets, kmeans)
train_dataset[0][0] # one example image flattened out into integers

tensor([63,  0, 76, 24,  0, 24, 24, 63, 72, 93,  0, 29,  0, 93, 93, 76, 89,  6,
         6,  6, 89,  6, 29, 24, 63,  4, 92, 26, 49, 59, 98, 49, 26, 92,  4, 63,
        29, 72, 29, 63, 63, 37, 50, 39, 46, 23, 44, 86, 60, 82, 24, 72, 63, 72,
         0,  0, 72, 29, 72, 76, 63, 63, 24, 72, 93, 63, 24, 72, 35, 35, 35, 35,
        35, 35, 22, 24, 29, 93, 76, 93, 72, 63, 29, 29,  0, 76, 72, 72, 72, 76,
        93, 63, 76, 29,  0, 76,  0, 76, 63, 24])

In [53]:
train_dataset[0][0].size()

torch.Size([100])

In [54]:
from mingpt.model import GPT, GPTConfig
block_size = 100
num_clusters = 100
mconf = GPTConfig(num_clusters, block_size,
                  n_layer=2, n_head=4, n_embd=128)
model = GPT(mconf)

07/10/2022 06:58:56 - INFO - mingpt.model -   number of parameters: 4.352000e+05


In [55]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=2, batch_size=32, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
epoch 1 iter 13: train loss 3.57745. lr 5.999290e-04:   2%|▏         | 14/782 [00:05<05:28,  2.33it/s]


KeyboardInterrupt: ignored

> /content/hacking-minGPT/mingpt/model.py(212)forward()
    210 
    211     def forward(self, idx, targets=None):
--> 212         b, t = idx.size()
    213         assert t <= self.block_size, "Cannot forward, model block size is exhausted."
    214 

ipdb> idx
tensor([[[63,  0, 76,  ..., 76, 24, 63]],

        [[93, 29, 24,  ..., 72, 24, 72]],

        [[93, 93, 63,  ..., 63, 63, 24]],

        ...,

        [[76, 24,  0,  ..., 93, 24, 72]],

        [[76,  0, 24,  ..., 24, 93, 29]],

        [[76, 24,  0,  ...,  0, 76,  0]]], dtype=torch.int32)
ipdb> idx.size()
torch.Size([32, 1, 100])
ipdb> q
